In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import inshallah as ins
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from geopy.distance import distance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [101]:
data = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")
vessels = pd.read_csv("../CSV/vessels.csv", sep="|")
test_og = pd.read_csv("../CSV/ais_test.csv")
cleaned = pd.read_csv("cleaned.csv", sep="|")

cleaned["time"] = pd.to_datetime(cleaned["time"])

cleaned.head()

,Unnamed: 0,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,...,delta_lat,delta_lon,delta_lat_cum,delta_lon_cum,speed_from_prev,dist_from_prev,dist_cum,delta_lat_lag_1,delta_lon_lag_1,volume
0,59132,2024-01-01 00:37:39,0.778417,0.0,0,4.956735,5,12-31 13:00,39.21009,9.10328,...,0.00002,-0.00002,0.00004,-0.00004,0.057871,0.002813,0.005626,0.00002,-0.00002,110205.333333
1,211080,2024-01-01 00:38:37,3.591888,10.1,0,2.600541,0,01-02 01:00,41.44703,-71.38101,...,-0.02108,-0.01730,-0.07102,-0.04084,18.653912,2.751452,8.635989,-0.04994,-0.02354,155176.608000
2,167178,2024-01-01 00:43:14,1.019272,11.8,0,4.066617,0,01-01 05:00,50.51531,1.00915,...,0.01129,0.03266,0.02888,0.15068,22.587815,2.635245,11.235098,0.01759,0.11802,111625.440000
3,23970,2024-01-01 00:46:32,4.532620,0.0,-1,3.403392,5,12-25 15:00,40.72221,29.48062,...,-0.00001,-0.00001,0.00000,-0.00001,0.004651,0.001395,0.002506,0.00001,0.00000,143377.920000
4,217313,2024-01-01 00:48:15,5.841617,10.9,0,0.733038,0,01-02 13:00,40.59254,18.32469,...,0.03224,-0.01957,0.08943,-0.05445,20.288515,3.944989,10.949607,0.05719,-0.03488,98435.064000


In [102]:
for i in range(2,6):
    cleaned[f"delta_lat_lag_{i}"] = cleaned[f"delta_lat_lag_{i-1}"].shift(1)
    cleaned[f"delta_lon_lag_{i}"] = cleaned[f"delta_lon_lag_{i-1}"].shift(1)

cleaned["delta_lat_cum_lag_1"] = cleaned["delta_lat_cum"].shift(1)
cleaned["delta_lon_cum_lag_1"] = cleaned["delta_lon_cum"].shift(1)

cleaned["day"] = cleaned["time"].apply(lambda t: t.day)
cleaned["month"] = cleaned["time"].apply(lambda t: t.month)
cleaned["hour"] = cleaned["time"].apply(lambda t: t.hour)

cleaned.dropna(inplace=True)
cleaned.head()

,Unnamed: 0,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,...,delta_lon_lag_3,delta_lat_lag_4,delta_lon_lag_4,delta_lat_lag_5,delta_lon_lag_5,delta_lat_cum_lag_1,delta_lon_cum_lag_1,day,month,hour
4,217313,2024-01-01 00:48:15,5.841617,10.9,0,0.733038,0,01-02 13:00,40.59254,18.32469,...,0.11802,-0.04994,-0.02354,0.00002,-0.00002,0.00000,-0.00001,1,1,0
5,11569,2024-01-01 00:50:50,2.638938,0.0,0,4.852015,5,12-31 18:30,-19.25026,146.83508,...,0.00000,0.01759,0.11802,-0.04994,-0.02354,0.08943,-0.05445,1,1,0
6,24128,2024-01-01 00:51:08,4.763004,7.6,-15,2.565634,0,01-16 01:00,43.80732,-8.65108,...,-0.03488,0.00001,0.00000,0.01759,0.11802,0.00000,0.00001,1,1,0
7,48839,2024-01-01 00:52:26,4.900885,0.1,0,1.570796,1,12-27 03:00,53.92163,7.80502,...,0.00000,0.05719,-0.03488,0.00001,0.00000,-0.00288,-0.11278,1,1,0
8,6693,2024-01-01 00:53:39,2.153736,0.0,128,2.356194,5,12-16 01:00,40.71467,29.46604,...,-0.07469,0.00001,0.00000,0.05719,-0.03488,0.00003,-0.00013,1,1,0


In [ ]:
n = 50
LAT_FEATURES = ["delta_time", "speed_from_prev", "dist_from_prev", 
                "delta_lat_lag_1", "delta_lat_cum_lag_1"]
LON_FEATURES = ["delta_time", "dist_cum", "speed_from_prev", "dist_from_prev", "delta_lon_lag_1", 
                "delta_lon_cum_lag_1", "hour", "day"]

for i in range(2,4):
    LAT_FEATURES.append(f"delta_lat_lag_{i}")
    LON_FEATURES.append(f"delta_lon_lag_{i}")
bruh = list(set(LAT_FEATURES + LON_FEATURES))

small = cleaned[cleaned["vesselId"].isin(cleaned["vesselId"].unique()[0:n])].copy()
mean, std = small[bruh].mean(), small[bruh].std()

small[bruh] = (small[bruh] - mean) / std

# Train-test split
X_lat, y_lat = small[LAT_FEATURES], small[["delta_lat", "delta_lat_cum"]]
X_lat_train, X_lat_test, y_lat_train, y_lat_test = train_test_split(X_lat, y_lat, test_size=0.1)
y_lat_test = y_lat_test["delta_lat"]

X_lon, y_lon = small[LON_FEATURES], small[["delta_lon",  "delta_lon_cum"]]
X_lon_train, X_lon_test, y_lon_train, y_lon_test = train_test_split(X_lon, y_lon, test_size=0.1)
y_lon_test = y_lon_test["delta_lon"]

X, y = small[list(set(LAT_FEATURES + LON_FEATURES))], small[["delta_lat", "delta_lon"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(y_test.shape, y_lat_test.shape, y_lon_test.shape)

(62884, 2) (62884,) (62884,)


In [104]:
latModel = XGBRegressor(n_estimators=3000, early_stopping_rounds=50)
latModel.fit(X_lat_train, y_lat_train, verbose=200, eval_set=[(X_lat_train, y_lat_train)])

lonModel = XGBRegressor(n_estimators=3000, early_stopping_rounds=50)
lonModel.fit(X_lon_train, y_lon_train, verbose=200, eval_set=[(X_lon_train, y_lon_train)])

lat_pred = latModel.predict(X_lat_test)[:, 0]
lon_pred = lonModel.predict(X_lon_test)[:, 0]
pred = np.concatenate((lat_pred.reshape((lat_pred.shape[0], 1)), lon_pred.reshape((lon_pred.shape[0], 1))), axis=1)

[0]	validation_0-rmse:12.82049
[200]	validation_0-rmse:12.23088
[400]	validation_0-rmse:11.93310
[600]	validation_0-rmse:11.66226
[800]	validation_0-rmse:11.42106
[1000]	validation_0-rmse:11.20257
[1200]	validation_0-rmse:11.00301
[1400]	validation_0-rmse:10.80959
[1600]	validation_0-rmse:10.63586
[1800]	validation_0-rmse:10.46543
[2000]	validation_0-rmse:10.31452
[2200]	validation_0-rmse:10.16037
[2400]	validation_0-rmse:10.01145
[2600]	validation_0-rmse:9.87479
[2800]	validation_0-rmse:9.73802
[2999]	validation_0-rmse:9.60895
[0]	validation_0-rmse:41.98522
[200]	validation_0-rmse:30.46168
[400]	validation_0-rmse:27.63492
[600]	validation_0-rmse:25.82552
[800]	validation_0-rmse:24.44819
[1000]	validation_0-rmse:23.21714
[1200]	validation_0-rmse:22.15173
[1400]	validation_0-rmse:21.25133
[1600]	validation_0-rmse:20.50219
[1800]	validation_0-rmse:19.77720
[2000]	validation_0-rmse:19.17065
[2200]	validation_0-rmse:18.62236
[2400]	validation_0-rmse:18.10218
[2600]	validation_0-rmse:17.584

In [105]:
latModel = RandomForestRegressor(n_estimators=50, max_depth=10, verbose=2)
latModel.fit(X_lat_train, y_lat_train)

lonModel = RandomForestRegressor(n_estimators=50, max_depth=10, verbose=2)
lonModel.fit(X_lon_train, y_lon_train)

lat_pred_rf = latModel.predict(X_lat_test)[:, 0]
lon_pred_rf = lonModel.predict(X_lon_test)[:, 0]
pred_rf = np.concatenate((lat_pred_rf.reshape((lat_pred.shape[0], 1)), lon_pred_rf.reshape((lon_pred.shape[0], 1))), axis=1)

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  6.5min


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50
building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  9.0min


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s


In [106]:
print("----- XGBoost ----- ")
print(f"Latitude MSE: {mean_squared_error(y_lat_test, lat_pred)}")
print(f"Longitude MSE: {mean_squared_error(y_lon_test, lon_pred)}")
print(f"Combined MSE: {mean_squared_error(y_test, pred)}\n")

print("----- Random Forest ----- ")
print(f"Latitude MSE: {mean_squared_error(y_lat_test, lat_pred_rf)}")
print(f"Longitude MSE: {mean_squared_error(y_lon_test, lon_pred_rf)}")
print(f"Combined MSE: {mean_squared_error(y_test, pred_rf)}\n")

----- XGBoost ----- 
Latitude MSE: 1.5695507354884795
Longitude MSE: 15.970320586018028
Combined MSE: 10.493224857260127

----- Random Forest ----- 
Latitude MSE: 0.8324462061535989
Longitude MSE: 11.735588759209598
Combined MSE: 7.96008847253491



[0.00362577 0.01715719 0.03658599 ... 0.00814429 0.02924506 0.00788658]


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s


In [107]:
LON_FEATURES = ["delta_time", "dist_cum", "speed_from_prev", "dist_from_prev", "delta_lon_lag_1", 
                "delta_lon_cum_lag_1", "hour", "day"]

for i in range(2,6):
    LON_FEATURES.append(f"delta_lon_lag_{i}")

X_lon, y_lon = small[LON_FEATURES], small[["delta_lon",  "delta_lon_cum"]]
X_lon_train, X_lon_test, y_lon_train, y_lon_test = train_test_split(X_lon, y_lon, test_size=0.1)
y_lon_test = y_lon_test["delta_lon"]

lonModel = RandomForestRegressor(n_estimators=50, max_depth=10, verbose=2)
lonModel.fit(X_lon_train, y_lon_train)

lon_pred_rf = lonModel.predict(X_lon_test)[:, 0]

print(f"Longitude MSE: {mean_squared_error(y_lon_test, lon_pred_rf)}")

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  8.7min


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s
